In [1]:
import os
import pandas as pd
import json
import numpy as np
from IPython.display import display, HTML

In [2]:
prj_root = os.getcwd()
dataset_folderpath= os.path.join(prj_root,'mitre_nav_reports')
ttp_list_corpus=[]

In [3]:
# extract all mitre ttps in each mitre report
for root, dirs, files in os.walk(dataset_folderpath):
    for filename in files:
        abs_filepath = os.path.join(root,filename)
        # print(abs_filepath)
        with open(abs_filepath,'r') as f:
            data = data = json.load(f)

        df = pd.DataFrame(data['techniques'])
        # df = df[df['score'].notna()]
        df = df.loc[df['score'].isin([1.0])]
        ttp_list= df['techniqueID'].tolist()

        ttp_list_corpus.append(' '.join(ttp_list))
    
#     break
    

In [ ]:
ttp_list_corpus

In [4]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

In [5]:
# src:https://github.com/kavgan/nlp-in-practice/blob/master/tf-idf/Keyword%20Extraction%20with%20TF-IDF%20and%20SKlearn.ipynb
from sklearn.feature_extraction.text import CountVectorizer
#instantiate CountVectorizer() 
cv=CountVectorizer(analyzer=lambda d: d.split(' ')) 
word_count_vector=cv.fit_transform(ttp_list_corpus)
terms = cv.get_feature_names()
#freq_count of all ttps
freq_count_dict = dict(zip(terms,word_count_vector.toarray().sum(axis=0)))
freq_count_dict

c:\users\yi sheng\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


{'T1001': 5,
 'T1001.001': 11,
 'T1001.002': 8,
 'T1001.003': 11,
 'T1003': 6,
 'T1003.001': 13,
 'T1003.002': 7,
 'T1003.004': 1,
 'T1003.005': 1,
 'T1005': 92,
 'T1007': 38,
 'T1008': 39,
 'T1010': 25,
 'T1011.001': 1,
 'T1012': 71,
 'T1014': 19,
 'T1016': 158,
 'T1016.001': 5,
 'T1018': 32,
 'T1020': 13,
 'T1021': 2,
 'T1021.001': 12,
 'T1021.002': 20,
 'T1021.003': 1,
 'T1021.004': 2,
 'T1021.005': 5,
 'T1021.006': 1,
 'T1025': 18,
 'T1027': 226,
 'T1027.001': 23,
 'T1027.002': 56,
 'T1027.003': 17,
 'T1027.004': 4,
 'T1027.005': 8,
 'T1027.006': 1,
 'T1029': 14,
 'T1030': 9,
 'T1033': 130,
 'T1036': 23,
 'T1036.001': 4,
 'T1036.003': 1,
 'T1036.004': 39,
 'T1036.005': 88,
 'T1036.006': 1,
 'T1036.007': 1,
 'T1037.001': 4,
 'T1037.004': 4,
 'T1037.005': 1,
 'T1039': 4,
 'T1040': 5,
 'T1041': 81,
 'T1046': 21,
 'T1047': 55,
 'T1048': 6,
 'T1048.003': 14,
 'T1049': 45,
 'T1052.001': 5,
 'T1053': 2,
 'T1053.002': 1,
 'T1053.003': 10,
 'T1053.005': 71,
 'T1055': 40,
 'T1055.001': 40,
 

In [6]:
# https://github.com/annts/tf-id_demystified/blob/master/tf_idf_demo.ipynb
# create a dataframe from a word matrix
def dtm2df(wm, feat_names):
    
    # create an index for each row
    doc_names = ['Doc{:d}'.format(idx) for idx, _ in enumerate(wm)]
    df = pd.DataFrame(data=wm.toarray(), index=doc_names,
                      columns=feat_names)
    return(df)

def idf2df(wm, feat_names):
  
    # create an index for each row
    doc_names = ['Doc{:d}'.format(idx) for idx, _ in enumerate(wm)]
    df = pd.DataFrame(data=wm, index=[0],
                      columns=feat_names)
    return(df)

In [7]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)
# tfidf_transformer.idf_

TfidfTransformer()

In [8]:
# idf_ attribute can be used to extract IDF values
# transpose the 1D IDF array to convert to a dataframe to make it easy to visualise
df_idf1 = idf2df(tfidf_transformer.idf_[:,np.newaxis].T ,terms)
df_idf1= df_idf1.sort_values(by =0, axis=1)
display(HTML(df_idf1.to_html()))

,T1105,T1082,T1071.001,T1027,T1059.003,T1083,T1057,T1070.004,T1140,T1016,T1547.001,T1033,T1573.001,T1106,T1113,T1112,T1056.001,T1005,T1036.005,T1543.003,T1041,T1518.001,T1059.001,T1053.005,T1012,T1132.001,T1027.002,T1047,T1204.002,T1074.001,T1124,T1562.001,T1218.011,T1059.005,T1095,T1049,T1555.003,T1573.002,T1055.001,T1055,T1008,T1036.004,T1007,T1566.001,T1486,T1497.001,T1564.001,T1569.002,T1497.003,T1553.002,T1018,T1120,T1560,T1087.001,T1070.006,T1548.002,T1135,T1490,T1102.002,T1059.004,T1115,T1119,T1071.004,T1560.003,T1090,T1574.002,T1489,T1547.009,T1010,T1070,T1036,T1027.001,T1125,T1123,T1518,T1571,T1055.012,T1046,T1059.007,T1021.002,T1014,T1204.001,T1614.001,T1568.002,T1560.001,T1025,T1574.001,T1564.003,T1059.006,T1027.003,T1543.001,T1566.002,T1562.004,T1485,T1091,T1129,T1059,T1029,T1102,T1218.007,T1218.010,T1069.001,T1048.003,T1071.003,T1068,T1070.001,T1020,T1090.003,T1003.001,T1090.001,T1497,T1102.001,T1555,T1021.001,T1210,T1185,T1560.002,T1552.001,T1564.004,T1001.001,T1203,T1136.001,T1087.002,T1001.003,T1071.002,T1069.002,T1053.003,T1134,T1134.001,T1529,T1030,T1056.004,T1480,T1570,T1546.015,T1218.005,T1001.002,T1559.002,T1222.002,T1539,T1090.002,T1567.002,T1205,T1110.001,T1071,T1027.005,T1056.002,T1543.004,T1572,T1134.002,T1222.001,T1554,T1114.001,T1559.001,T1197,T1496,T1003.002,T1189,T1132.002,T1080,T1546.003,T1547.004,T1217,T1505.003,T1219,T1552.004,T1542.003,T1568,T1003,T1055.004,T1048,T1561.002,T1078,T1573,T1087.003,T1069,T1055.002,T1052.001,T1040,T1555.004,T1001,T1104,T1110,T1021.005,T1016.001,T1620,T1036.001,T1027.004,T1039,T1574.006,T1614,T1480.001,T1037.001,T1037.004,T1555.001,T1102.003,T1133,T1132,T1561.001,T1078.003,T1569.001,T1078.002,T1564.005,T1555.005,T1565.002,T1548.003,T1553.006,T1543.002,T1611,T1609,T1553.004,T1564.006,T1546.011,T1567,T1562.006,T1574,T1564,T1553.001,T1547,T1552.002,T1546.010,T1547.015,T1195.002,T1542.001,T1098.004,T1114.002,T1059.002,T1134.004,T1176,T1055.003,T1190,T1622,T1482,T1484.001,T1078.001,T1531,T1546.012,T1218.004,T1070.005,T1218.002,T1110.003,T1070.003,T1053,T1559,T1546.004,T1505.004,T1137.001,T1055.011,T1556,T1055.013,T1568.001,T1556.003,T1610,T1021.004,T1056,T1550.002,T1497.002,T1495,T1491.001,T1499,T1547.006,T1547.008,T1547.012,T1547.013,T1021,T1221,T1548.001,T1092,T1564.010,T1098,T1090.004,T1003.004,T1070.002,T1566,T1615,T1072,T1564.009,T1565.001,T1053.002,T1003.005,T1021.003,T1601.001,T1598.003,T1598.002,T1588.003,T1574.013,T1574.010,T1055.015,T1021.006,T1074,T1036.006,T1036.007,T1037.005,T1613,T1011.001,T1055.005,T1027.006,T1036.003,T1134.003,T1564.002,T1552.003,T1218.009,T1552,T1550.003,T1550,T1548.004,T1218.012,T1220,T1547.014,T1547.002,T1498,T1546.009,T1499.004,T1546.002,T1505.001,T1505.002,T1543,T1552.005,T1087,T1218.001,T1202,T1563.002,T1562.009,T1110.002,T1562,T1110.004,T1111,T1127.001,T1542.002,T1557,T1556.004,T1556.002,T1556.001,T1137.006,T1187,T1555.002,T1195.001,T1201,T1213,T1647
0,1.567054,1.654383,1.797191,1.801587,1.86995,1.874679,2.011601,2.067481,2.102573,2.157632,2.189584,2.351339,2.472946,2.508038,2.517006,2.601564,2.67266,2.693937,2.7379,2.79572,2.819817,2.922472,2.922472,2.949871,2.949871,2.978041,3.183485,3.201185,3.219203,3.219203,3.237553,3.256245,3.294711,3.294711,3.334716,3.397895,3.419874,3.442347,3.512965,3.512965,3.537657,3.537657,3.562975,3.588951,3.615619,3.643018,3.671189,3.700176,3.700176,3.730029,3.730029,3.730029,3.760801,3.792549,3.792549,3.825339,3.825339,3.825339,3.894332,3.894332,3.894332,3.894332,3.9307,3.9307,3.9307,3.9307,3.96844,3.96844,3.96844,4.007661,4.048483,4.048483,4.091042,4.091042,4.091042,4.135494,4.135494,4.135494,4.182014,4.182014,4.230804,4.230804,4.282098,4.282098,4.282098,4.282098,4.282098,4.336165,4.336165,4.336165,4.393323,4.393323,4.393323,4.393323,4.393323,4.453948,4.453948,4.518486,4.518486,4.518486,4.518486,4.518486,4.518486,4.518486,4.518486,4.518486,4.587479,4.587479,4.587479,4.587479,4.587479,4.661587,4.661587,4.661587,4.74163,4.74163,4.74163,4.74163,4.74163,4.74163,4.74163,4.74163,4.74163,4.74163,4.828641,4.

In [24]:
# Using The min-max feature scaling
# copy the data
df_min_max_scaled = df_idf1.copy().T
dataf=round(((df_min_max_scaled-df_min_max_scaled.min())/(df_min_max_scaled.max()-df_min_max_scaled.min()))*2,2)
dataf.index.name="TTP"
dataf.columns=['weightage']
# dataf['count'] = pd.Series(freq_count_dict)
# dataf.reset_index(inplace=True)
# dataf = dataf.rename(columns = {'index':'TTP'})



dataf

,weightage
TTP,
T1105,0.00
T1082,0.04
T1071.001,0.09
T1027,0.09
T1059.003,0.12
...,...
T1555.002,2.00
T1195.001,2.00
T1201,2.00


In [26]:
dataf.to_dict('dict')['weightage']

{'T1105': 0.0,
 'T1082': 0.04,
 'T1071.001': 0.09,
 'T1027': 0.09,
 'T1059.003': 0.12,
 'T1083': 0.12,
 'T1057': 0.18,
 'T1070.004': 0.2,
 'T1140': 0.22,
 'T1016': 0.24,
 'T1547.001': 0.25,
 'T1033': 0.32,
 'T1573.001': 0.36,
 'T1106': 0.38,
 'T1113': 0.38,
 'T1112': 0.42,
 'T1056.001': 0.45,
 'T1005': 0.45,
 'T1036.005': 0.47,
 'T1543.003': 0.49,
 'T1041': 0.5,
 'T1518.001': 0.55,
 'T1059.001': 0.55,
 'T1053.005': 0.56,
 'T1012': 0.56,
 'T1132.001': 0.57,
 'T1027.002': 0.65,
 'T1047': 0.66,
 'T1204.002': 0.67,
 'T1074.001': 0.67,
 'T1124': 0.67,
 'T1562.001': 0.68,
 'T1218.011': 0.7,
 'T1059.005': 0.7,
 'T1095': 0.71,
 'T1049': 0.74,
 'T1555.003': 0.75,
 'T1573.002': 0.76,
 'T1055.001': 0.78,
 'T1055': 0.78,
 'T1008': 0.79,
 'T1036.004': 0.79,
 'T1007': 0.8,
 'T1566.001': 0.81,
 'T1486': 0.82,
 'T1497.001': 0.84,
 'T1564.001': 0.85,
 'T1569.002': 0.86,
 'T1497.003': 0.86,
 'T1553.002': 0.87,
 'T1018': 0.87,
 'T1120': 0.87,
 'T1560': 0.88,
 'T1087.001': 0.9,
 'T1070.006': 0.9,
 'T1548.

In [ ]:
tfIdfVectorizer=TfidfVectorizer(use_idf=True,analyzer=lambda d: d.split(' '))
tfIdf = tfIdfVectorizer.fit_transform(ttp_list_corpus)
terms = tfIdfVectorizer.get_feature_names()
terms
# df = pd.DataFrame(tfIdf[0].T.todense(), index=tfIdfVectorizer.get_feature_names(), columns=["TF-IDF"])
# df = df.sort_values('TF-IDF', ascending=False)
# print (df.head(25))

In [ ]:
# idf_ attribute can be used to extract IDF values
# transpose the 1D IDF array to convert to a dataframe to make it easy to visualise
df_idf = idf2df(tfIdfVectorizer.idf_[:,np.newaxis].T ,terms)
display(HTML(df_idf.to_html()))

In [ ]:
df_idf= df_idf.sort_values(by =0, axis=1)
df_idf

In [ ]:
df_idf.reset_index(drop=True)
df_idf
# df_idf.to_csv('weightage.csv', index=False)

In [ ]:
# from sklearn.preprocessing import Normalizer
# df_idf.iloc[:,:] = Normalizer(norm='max').fit_transform(df_idf)
# display(HTML(df_idf.to_html()))

In [ ]:
# Using The min-max feature scaling
# copy the data
df_min_max_scaled = df_idf.copy().T
dataf=round(((df_min_max_scaled-df_min_max_scaled.min())/(df_min_max_scaled.max()-df_min_max_scaled.min()))*2,2)
dataf.index.name="TTP"
dataf['count'] = pd.Series(freq_count_dict)
dataf.reset_index(inplace=True)
dataf = dataf.rename(columns = {'index':'TTP'})

dataf.columns=['TTP','weight','count']

dataf
# dataf.to_csv('idf.csv',index=False,header=True)

In [ ]:
dataf.to_json('idf.json',orient='table',index=False)